# A very simple Recommendation system 

Only suggests similar titles based on the genre similarity 

## Imports

In [1]:
import pandas as pd
import numpy as np
import pickle 

import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

## Loading dataset

Download from [kaggle link](https://www.kaggle.com/snehaanbhawal/netflix-tv-shows-and-movie-list)

Dropping all the colums except only the Genre and Title

In [2]:
df = pd.read_csv("netflix_list.csv")

In [3]:
df.drop(["imdb_id","popular_rank", "runtime", "type","language", "endYear","certificate", "rating", "startYear","episodes","orign_country","isAdult","image_url","summary","cast","numVotes","plot"], axis=1, inplace=True)
df.dropna(inplace = True)
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,title,genres
0,Lucifer,"Crime,Drama,Fantasy"
1,Army of the Dead,"Action,Crime,Horror"
2,The Kominsky Method,"Comedy,Drama"
3,Friends,"Comedy,Romance"
4,Ragnarok,"Action,Drama,Fantasy"


In [5]:
df.isnull().sum()

title     0
genres    0
dtype: int64

## Some Processing data

In [6]:
df.genres = [x.replace(","," ") for x in df.genres]

In [7]:
df.genres.str.split(expand=True).stack().value_counts()

Drama          2848
Comedy         2176
Action         1233
Documentary    1020
Crime           998
Animation       942
Adventure       907
Thriller        683
Romance         671
Mystery         481
Horror          426
Fantasy         384
Family          376
Biography       281
Sci-Fi          239
History         226
Reality-TV      199
Music           173
Short           148
Sport           137
\N               87
Musical          52
War              49
Talk-Show        38
Game-Show        35
Western          25
News             21
Film-Noir        11
dtype: int64

In [8]:
# v = TfidfVectorizer()
v = CountVectorizer()
x = v.fit_transform(df["genres"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df1 = pd.concat([df, df1], axis=1)
df1.drop(["genres","title"], axis = 1, inplace=True) 

In [9]:
df1.head()

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,...,romance,sci,short,show,sport,talk,thriller,tv,war,western
0,0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df1.head()

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,...,romance,sci,short,show,sport,talk,thriller,tv,war,western
0,0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
import scipy
mat = scipy.sparse.csr_matrix(df1.values)
from sklearn.metrics.pairwise import sigmoid_kernel
sig1 = sigmoid_kernel(mat,mat)

In [12]:
index = pd.Series(df.index, index = df.title).drop_duplicates()

In [13]:
def recommend(title, sig=sig1):
    idx = index[title]
    scores = list(enumerate(sig1[idx]))
    scores = sorted(scores, key=lambda x:x[1], reverse=True)
    scores= scores[1:11]
    mov_index = [i[0] for i in scores]
    return df.iloc[mov_index]

In [14]:
recommend("Ragnarok")

,title,genres
112,Teen Wolf,Action Drama Fantasy
312,Shadowhunters: The Mortal Instruments,Action Drama Fantasy
464,Warrior Nun,Action Drama Fantasy
590,Word of Honor,Action Drama Fantasy
624,The School for Good and Evil,Action Drama Fantasy
704,Sijipeuseu: The Myth,Action Drama Fantasy
924,Aseudal Yeondaegi,Action Drama Fantasy
992,Yin-Yang Master I,Action Drama Fantasy
1476,Riaru onigokko,Action Drama Fantasy
1511,Mugen no jûnin,Action Drama Fantasy


	title	genres
112	Teen Wolf	Action Drama Fantasy
312	Shadowhunters: The Mortal Instruments	Action Drama Fantasy
464	Warrior Nun	Action Drama Fantasy
590	Word of Honor	Action Drama Fantasy
624	The School for Good and Evil	Action Drama Fantasy
704	Sijipeuseu: The Myth	Action Drama Fantasy
924	Aseudal Yeondaegi	Action Drama Fantasy
992	Yin-Yang Master I	Action Drama Fantasy
1476	Riaru onigokko	Action Drama Fantasy
1511	Mugen no jûnin	Action Drama Fantasy

In [15]:

# math.ceil(1.6)